In [7]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 512

class_vocab = ['body-building', 'boxing', 'calesthenics', 'cycling', 'swimming', 'yoga']

In [1]:
from tensorflow.keras.models import load_model

sequence_model = load_model( 'models/best_model_30_11.keras')

In [2]:
sequence_model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_21 (InputLayer)       [(None, 20, 2048)]           0         []                            
                                                                                                  
 input_22 (InputLayer)       [(None, 20)]                 0         []                            
                                                                                                  
 lstm_2 (LSTM)               (None, 20, 32)               266368    ['input_21[0][0]',            
                                                                     'input_22[0][0]']            
                                                                                                  
 lstm_3 (LSTM)               (None, 16)                   3136      ['lstm_2[0][0]']        

In [9]:
from tensorflow_docs.vis import embed
from tensorflow import keras
import numpy as np

import cv2
import os

IMG_SIZE = 224

def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]

def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

from tensorflow.keras.applications import InceptionV3, VGG16
from tensorflow.keras import Input, Model

def build_feature_extractor(model_name="inception_v3"):
    if model_name == "inception_v3":
        feature_extractor = InceptionV3(
            weights="imagenet",
            include_top=False,
            pooling="avg",
            input_shape=(IMG_SIZE, IMG_SIZE, 3),
        )
        preprocess_input = keras.applications.inception_v3.preprocess_input
    elif model_name == "vgg16":
        feature_extractor = VGG16(
            weights="imagenet",
            include_top=False,
            pooling="avg",
            input_shape=(IMG_SIZE, IMG_SIZE, 3),
        )
        preprocess_input = keras.applications.vgg16.preprocess_input
    else:
        raise ValueError("Invalid model name. Supported names: 'inception_v3', 'vgg16'")

    inputs = Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return Model(inputs, outputs, name=f"{model_name}_feature_extractor")

# To use InceptionV3
inception_feature_extractor = build_feature_extractor(model_name="inception_v3")

# To use VGG16
vgg16_feature_extractor = build_feature_extractor(model_name="vgg16")

def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = vgg16_feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask

import os

def sequence_prediction(path):
    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

In [10]:
import os

path = 'data'
dataset_path = os.listdir( path)

label_types = os.listdir( path)
print (label_types)

['body-building', 'boxing', 'calesthenics', 'cycling', 'swimming', 'yoga']


In [11]:
import pandas as pd

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir( path + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str( path + '/' +item) + '/' + room))

# Build a dataframe
data = pd.DataFrame( data=rooms, columns=['tag', 'video_name'])
print(data.head())
print(data.tail())

             tag                                         video_name
0  body-building  C:/Users/aminb/Downloads/RNN/data/body-buildin...
1  body-building  C:/Users/aminb/Downloads/RNN/data/body-buildin...
2  body-building  C:/Users/aminb/Downloads/RNN/data/body-buildin...
3  body-building  C:/Users/aminb/Downloads/RNN/data/body-buildin...
4  body-building  C:/Users/aminb/Downloads/RNN/data/body-buildin...
       tag                                         video_name
1633  yoga  C:/Users/aminb/Downloads/RNN/data/yoga/Young H...
1634  yoga  C:/Users/aminb/Downloads/RNN/data/yoga/Young P...
1635  yoga  C:/Users/aminb/Downloads/RNN/data/yoga/Young W...
1636  yoga  C:/Users/aminb/Downloads/RNN/data/yoga/Young W...
1637  yoga  C:/Users/aminb/Downloads/RNN/data/yoga/Young W...


In [12]:
import numpy as np

test_video = np.random.choice( data["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction( test_video)

Test video path: C:/Users/aminb/Downloads/RNN/data/cycling/Cycling stock photos, royalty-free images, vectors, video_63.mp4
1/1 [==============================] - 6s 6s/step
  cycling: 81.23%
  swimming: 15.34%
  calesthenics:  1.58%
  body-building:  0.95%
  boxing:  0.77%
  yoga:  0.13%


In [13]:
from IPython.display import HTML

html = "<video alt='test' width='520' height='440' controls><source src='" + test_video[29:] + "' type='video/mp4' style='height:300px;width:300px'></video>"

HTML( html)